In [1]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import operator
import pickle

import src.fusiontools as fusiontools
import src.scoring as scoring
import src.postprocess as postprocess

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path_to_results = 'results/'

# Load best models

In [16]:
# Load best recency model
recency_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
with open(recency_rank_dic_path, 'rb') as infile:
    recency_ranks = pickle.load(infile)
    
# Load best knn model
knn_rank_dic_path = 'ranks_test/knn-k-50-rec-all'
with open(knn_rank_dic_path, 'rb') as infile:
    knn_ranks = pickle.load(infile)

# Load best bayesian model
bayes_rank_dic_path = 'ranks_test/bayesian_results_0-35785'
with open(bayes_rank_dic_path, 'rb') as infile:
    bayes_ranks = pickle.load(infile)
    
# # Load best graph_through_words model
words_rank_dic_path = 'ranks_test/resjo_pickle_318'
with open(words_rank_dic_path, 'rb') as infile:
    words_ranks = pickle.load(infile)

# Load best graph_through_words model
centroids_rank_dic_path = 'ranks_test/centroids_dict_test'
with open(centroids_rank_dic_path, 'rb') as infile:
    centroids_ranks = pickle.load(infile)
    
# Load best word2vec model
word2vec_rank_dic_path = 'ranks_test/centroids_w2v_sw0_test'
with open(word2vec_rank_dic_path, 'rb') as infile:
    word2vec_ranks = pickle.load(infile)
    
# Load best twidf model
twidf_knn_rank_dic_path = 'ranks_test/twidf-knn-k-50-rec'
with open(twidf_knn_rank_dic_path, 'rb') as infile:
    twidf_knn_ranks = pickle.load(infile)
    
# Load best twidf sender model
twidf_knn_sender_rank_dic_path = 'ranks_test/twidf-knn-k-senders-50-rec'
with open(twidf_knn_sender_rank_dic_path, 'rb') as infile:
    twidf_knn_sender_ranks = pickle.load(infile)
    
# Load best tree sender model
tree_sender_rank_dic_path = 'ranks_test/trees-tree-nb-20-200'
with open(tree_sender_rank_dic_path, 'rb') as infile:
    tree_sender_ranks = pickle.load(infile)
    
# From model
from_rank_dic_path = 'ranks_test/frompredictions'
with open(from_rank_dic_path, 'rb') as infile:
    from_ranks = pickle.load(infile)

# Discard tail of recipients

In [17]:
nb_kept_recips = 100
knn_ranks_cropped = fusiontools.keep_only_max_recips(knn_ranks, nb_kept_recips)
recency_ranks_cropped = fusiontools.keep_only_max_recips(recency_ranks, nb_kept_recips)
bayes_ranks_cropped = fusiontools.keep_only_max_recips(bayes_ranks, nb_kept_recips)
centroids_ranks_cropped = fusiontools.keep_only_max_recips(centroids_ranks, nb_kept_recips)
words_ranks_cropped = fusiontools.keep_only_max_recips(words_ranks, nb_kept_recips)
twidf_knn_sender_ranks_cropped = fusiontools.keep_only_max_recips(twidf_knn_sender_ranks, nb_kept_recips)
tree_sender_ranks_cropped = fusiontools.keep_only_max_recips(tree_sender_ranks, nb_kept_recips)
from_ranks_cropped = fusiontools.keep_only_max_recips(from_ranks, nb_kept_recips)

# Prepare models and weights

In [32]:
# models = [knn_ranks, recency_ranks]
# weights = [0.20716, 0.39316]

# models = [bayes_100, recency_ranks]
# weights = [0.34218, 0.39316]



# models = [bayes_ranks_cropped, recency_ranks_cropped, tree_sender_ranks_cropped]
# weights = [0.35785, 0.39316, 0.29137]


# models = [bayes_ranks_cropped, recency_ranks_cropped, words_ranks_cropped]
# weights = [0.35785, 0.39316, 0.33785]

# models = [bayes_ranks_cropped, recency_ranks_cropped, words_ranks_cropped, knn_ranks_cropped]
# weights = [0.35785*0.6, 0.39316*0.5, 0.33785*0.6, 0.20716*0.2]

# models = [knn_ranks_cropped, recency_ranks_cropped, bayes_ranks_cropped, centroid_ranks_cropped, words_ranks_cropped]
#val 0.5, 1, 0.5, 1, 0.5,
# models = [bayes_ranks_cropped, centroids_ranks_cropped, recency_ranks_cropped, knn_ranks_cropped, twidf_knn_sender_ranks_cropped, tree_sender_ranks_cropped]
# weights = [0.35785, 0.24153, 0.39316, 0.20716, 0.33785, 0.32109, 0.34583 ]

models = [bayes_ranks_cropped, centroids_ranks_cropped, recency_ranks_cropped, knn_ranks_cropped, tree_sender_ranks_cropped, from_ranks_cropped]
weights = [0.35785, 0.24153, 0.39316, 0.20716, 0.33785, 0.34583, 100 ]

#Fast check that dictionnaries are the right lenght
for model in models:
    print(len(model))

2362
2362
2362
2362
2362
521


# Merge models and save results!

In [33]:
reranking_coeff = 7
fusion_dic = fusiontools.reciprocal_rerank(models, reranking_coeff,  weights=weights)
# print(fusion_dic)

            
                

In [82]:
postprocess.write_results_ranked(manual_dic, path_to_results, 'add-email-in-content-to-bayes-recency-knn-tfidf.csv')

In [34]:
postprocess.write_results_ranked(fusion_dic, path_to_results, 'fusion-weighted-reranking-7-bayes-recency-tree-centroids-add-from-weight-100.csv')

In [83]:
189815 in emails_in_content.keys()

True